## Instalar librerias necesarias

In [1]:
pip install psycopg2-binary pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import psycopg2
import pandas as pd

# Paso 1: Crear base y tabla para CVEs del NIST
def create_database_and_table():
    try:
        conn = psycopg2.connect(
            user="postgres",
            password="postgres",
            host="localhost",
            port="5432"
        )
        conn.autocommit = True
        cursor = conn.cursor()

        cursor.execute("SELECT datname FROM pg_database WHERE datname='nist_vulnerabilities';")
        if not cursor.fetchone():
            cursor.execute("CREATE DATABASE nist_vulnerabilities;")
            print(" Base de datos 'nist_vulnerabilities' creada correctamente.")
        else:
            print(" La base de datos 'nist_vulnerabilities' ya existe.")

        cursor.close()
        conn.close()

        # Conexión a la nueva base
        conn = psycopg2.connect(
            database="nist_vulnerabilities",
            user="postgres",
            password="postgres",
            host="localhost",
            port="5432"
        )
        cursor = conn.cursor()

        cursor.execute("""
            CREATE TABLE IF NOT EXISTS nist_cve_data (
                id SERIAL PRIMARY KEY,
                cve_id VARCHAR,
                published TIMESTAMP,
                last_modified TIMESTAMP,
                description TEXT,
                severity VARCHAR,
                cvss_score FLOAT,
                vector VARCHAR,
                exploitability_score FLOAT,
                impact_score FLOAT
            );
        """)
        print(" Tabla 'nist_cve_data' creada correctamente.")
        conn.commit()
        cursor.close()
        conn.close()

    except Exception as e:
        print(f" Error al crear la base o la tabla: {e}")

# Paso 2: Insertar 10 CVEs reales desde el CSV
def insert_real_cves():
    try:
        df = pd.read_csv("tratados.csv", parse_dates=["published", "last_modified"])
        df = df.dropna(subset=["cvss_score"])  # Asegurar registros válidos
        df_10 = df.head(10)

        conn = psycopg2.connect(
            database="nist_vulnerabilities",
            user="postgres",
            password="postgres",
            host="localhost",
            port="5432"
        )
        cursor = conn.cursor()

        for _, row in df_10.iterrows():
            cursor.execute("""
                INSERT INTO nist_cve_data (
                    cve_id, published, last_modified, description,
                    severity, cvss_score, vector, exploitability_score, impact_score
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
            """, (
                row['cve_id'],
                row['published'],
                row['last_modified'],
                row['description'],
                row['severity'],
                row['cvss_score'],
                row['vector'],
                row['exploitability_score'],
                row['impact_score']
            ))

        conn.commit()
        print(" 10 vulnerabilidades insertadas correctamente.")
        cursor.close()
        conn.close()

    except Exception as e:
        print(f" Error al insertar CVEs: {e}")

# Paso 3: Mostrar registros insertados
def mostrar_registros():
    try:
        conn = psycopg2.connect(
            database="nist_vulnerabilities",
            user="postgres",
            password="postgres",
            host="localhost",
            port="5432"
        )
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM nist_cve_data LIMIT 10;")
        rows = cursor.fetchall()

        print("\n Registros en la tabla:")
        for row in rows:
            print(row)

        cursor.close()
        conn.close()
    except Exception as e:
        print(f" Error al mostrar registros: {e}")

# Ejecución total
if __name__ == "__main__":
    create_database_and_table()
    insert_real_cves()
    mostrar_registros()


 La base de datos 'nist_vulnerabilities' ya existe.
 Tabla 'nist_cve_data' creada correctamente.
 10 vulnerabilidades insertadas correctamente.

 Registros en la tabla:
(1, 'CVE-2006-4994', datetime.datetime(2006, 9, 26, 2, 7), datetime.datetime(2024, 11, 21, 0, 17, 26, 117000), 'multiple unquoted windows search path vulnerabilities in apache friends xampp 1.5.2 might allow local users to gain privileges via a malicious program file in %systemdrive%, which is run when xampp attempts to execute (1) filezillaserver.exe, (2) mysqld-nt.exe, (3) perl.exe, or (4) xamppcontrol.exe with an unquoted "program files" pathname.', 'medium', 4.6, 'AV:L/AC:L/Au:N/C:P/I:P/A:P', 3.9, 6.4)
(2, 'CVE-2006-4694', datetime.datetime(2006, 9, 27, 19, 7), datetime.datetime(2024, 11, 21, 0, 16, 33, 947000), 'unspecified vulnerability in powerpoint in microsoft office 2000, office xp and office 2003 allows user-assisted attackers to execute arbitrary code via a crafted record in a ppt file, as exploited by malwa